# Collaboration


- [Item 49: Write Docstrings for Every Function, Class, and Module](#Item-49:-Write-Docstrings-for-Every-Function,-Class,-and-Module)

## Item 49: Write Docstrings for Every Function, Class, and Module

In [ ]:
"""
Documentation from a program's source code is
directly accessible as the program runs
"""

def palindrome(word):
    """Return True if the given word"""
    return word == word[::-1]

print(repr(palindrome.__doc__))
help(palindrome)

### Documenting Modules

Each module should have a top-level docstring. This is a string literal that is the first statement in a source file. It should use three double quotes ("""). The goal of this docstring is to introduce the module and its contents.

The first line of the docstring should be a single sentence describing the module's purpose. The paragraphs that follow should contain the details that all users of the module should know about its operation. The module docstring is also a jumping-off point where you can highlight important classes and functions found in the module.

Example below:

In [ ]:
# words.py
# !/usr/bin/env python3
"""Library for testing words for various linguistic patterns.

Testing how words relate to each other can be tricky sometimes! This module provides easy ways to determine when words you’ve found have special properties.

Available functions:
- palindrome: Determine if a word is a palindrome.
- check_anagram: Determine if two words are anagrams.
...
"""

# ...

### Documenting Classes

In [ ]:
class Player(object):
    """Represents a player of the game.
    
    Subclasses may override the 'tick' method to provide
    custom animations for the player's movement depending
    on their power level, etc.
    
    Public attributes:
    - power: Unused power-ups (float between 0 and 1).
    - coins: Coins found during the level (integer).
    """
    
    # ...

### Documenting Functions

Any return values should be mentioned. Any exceptions that callers must handle as part of the function's interface should be explained.

In [ ]:
def find_anagrams(word, dictionary):
    """Find all anagrams for a word.
    
    This function only runs as fast as the test for
    membership in the 'dictionary' container. It will
    be slow if the dictionary is a list and fast if
    it's a set.
    
    Args:
        word: String of the target word.
        dictionary: Container with all strings that
            are known to be actual words.
            
    Return:
        List of anagrams that were found. Empty if
        none were found.
    """
    
    # ...

### Things to Remember

- Write documentation for every module, class, and function using docstrings. Keep them up to date as your code changes.
- For modules: Introduce the contents of the module and any important classes or functions all users should know about.
- For classes: Document behavior, important attributes, and subclass behavior in the docstring following the *class* statement.
- For functions and methods: Document every argument, returned value, raised exception, adn other behaviors in the docstring following the *def* statement.